In [1]:
import pandas as pd
import os
from datetime import date
import numpy as np

In [2]:
if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

dir_path = os.getcwd()

In [3]:
current_season = int(input('What is the current season?'))

In [4]:
today = date.today()

df_train = pd.read_csv(fr'{dir_path}\{current_season}_{current_season + 1}_Clean_Data\{current_season}_{current_season + 1}_{today.strftime("%b%d")}_historical_data.csv', index_col = 0)

df_pred = pd.read_csv(fr'{dir_path}\{current_season}_{current_season + 1}_Clean_Data\{current_season}_{current_season + 1}_{today.strftime("%b%d")}_matchweek_data.csv', index_col = 0)

In [5]:
df = pd.concat([df_train,df_pred])
df = df.sort_values(by = ['Club','Season','Date',])
df=df.reset_index(drop = True)
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
## RESULT ENCODING ##
def encode_result(x):
    if x['Result'] == 'W':
        val= 2
    elif x['Result'] == 'D':
        val= 1
    else:
        val = 0
    return val

df['Result'] = df.apply(encode_result, axis = 1) 

#POINTS DIFF#
df['Points_Diff'] = (df['Pts_x']/df['Pl_x'] - df['Pts_y']/df['Pl_y'])/3


##SPI Diff##
df['SPI_Diff'] = df['spi_x'] - df['spi_y']
df['SPI_Diff'] = (df['SPI_Diff'] - df.groupby(['Season'])['SPI_Diff'].transform(min))/(df.groupby(['Season'])['SPI_Diff'].transform(max) - df.groupby(['Season'])['SPI_Diff'].transform(min))
df['Off_Diff'] = df['off_x'] - df['off_y']
df['Off_Diff'] = (df['Off_Diff'] - df.groupby(['Season'])['Off_Diff'].transform(min))/(df.groupby(['Season'])['Off_Diff'].transform(max) - df.groupby(['Season'])['Off_Diff'].transform(min))
df['Def_Diff'] = df['def_x'] - df['def_y']
df['Def_Diff'] = (df['Def_Diff'] - df.groupby(['Season'])['Def_Diff'].transform(min))/(df.groupby(['Season'])['Def_Diff'].transform(max) - df.groupby(['Season'])['Def_Diff'].transform(min))

##FORM##
df['Form_Diff'] = (df['Form_x'] - df['Form_y'])/15

# Creating form for goals and goals conceded and expected values for last 5 games
df['Avg_GF_last_5'] = df.groupby(['Club','Season'])['GF_x'].shift(1)
df['Avg_GF_last_5'] = df.groupby(['Club','Season'])['Avg_GF_last_5'].rolling(5).mean().reset_index([0,1],drop=True)
df['Avg_GA_last_5'] = df.groupby(['Club','Season'])['GA_x'].shift(1)
df['Avg_GA_last_5'] = df.groupby(['Club','Season'])['Avg_GA_last_5'].rolling(5).mean().reset_index([0,1],drop=True)
df['Avg_xG_last_5'] = df.groupby(['Club','Season'])['xG_x'].shift(1)
df['Avg_xG_last_5'] = df.groupby(['Club','Season'])['Avg_xG_last_5'].rolling(5).mean().reset_index([0,1],drop=True)
df['Avg_xGA_last_5'] = df.groupby(['Club','Season'])['xGA_x'].shift(1)
df['Avg_xGA_last_5'] = df.groupby(['Club','Season'])['Avg_xGA_last_5'].rolling(5).mean().reset_index([0,1],drop=True)
df['Avg_Poss_last_5'] = df.groupby(['Club','Season'])['Poss_x'].shift(1)
df['Avg_Poss_last_5'] = df.groupby(['Club','Season'])['Avg_Poss_last_5'].rolling(5).mean().reset_index([0,1],drop=True)

#Standardisation
df['Avg_GF_last_5'] = (df['Avg_GF_last_5'] - df.groupby(['Season'])['Avg_GF_last_5'].transform(min))/(df.groupby(['Season'])['Avg_GF_last_5'].transform(max) - df.groupby(['Season'])['Avg_GF_last_5'].transform(min))
df['Avg_GA_last_5'] = (df['Avg_GA_last_5'] - df.groupby(['Season'])['Avg_GA_last_5'].transform(min))/(df.groupby(['Season'])['Avg_GA_last_5'].transform(max) - df.groupby(['Season'])['Avg_GA_last_5'].transform(min))
df['Avg_xG_last_5'] = (df['Avg_xG_last_5'] - df.groupby(['Season'])['Avg_xG_last_5'].transform(min))/(df.groupby(['Season'])['Avg_xG_last_5'].transform(max) - df.groupby(['Season'])['Avg_xG_last_5'].transform(min))
df['Avg_xGA_last_5'] = (df['Avg_xGA_last_5'] - df.groupby(['Season'])['Avg_xGA_last_5'].transform(min))/(df.groupby(['Season'])['Avg_xGA_last_5'].transform(max) - df.groupby(['Season'])['Avg_xGA_last_5'].transform(min))
df['Avg_Poss_last_5'] = (df['Avg_Poss_last_5'] - df.groupby(['Season'])['Avg_Poss_last_5'].transform(min))/(df.groupby(['Season'])['Avg_Poss_last_5'].transform(max) - df.groupby(['Season'])['Avg_Poss_last_5'].transform(min))

##SEASON STATS##
df['Avg_GF_season'] = df.groupby(['Club','Season'])['GF_x'].shift(1)
df['Avg_GF_season'] = df.groupby(['Season'])['Avg_GF_season'].expanding(1).mean().reset_index([0],drop=True)
df['Avg_GA_season'] = df.groupby(['Club','Season'])['GA_x'].shift(1)
df['Avg_GA_season'] = df.groupby(['Season'])['Avg_GA_season'].expanding(1).mean().reset_index([0],drop=True)
df['Avg_xG_season'] = df.groupby(['Club','Season'])['xG_x'].shift(1)
df['Avg_xG_season'] = df.groupby(['Season'])['Avg_xG_season'].expanding(1).mean().reset_index([0],drop=True)
df['Avg_xGA_season'] = df.groupby(['Club','Season'])['xGA_x'].shift(1)
df['Avg_xGA_season'] = df.groupby(['Season'])['Avg_xGA_season'].expanding(1).mean().reset_index([0],drop=True)
df['Avg_Poss_season'] = df.groupby(['Club','Season'])['Poss_x'].shift(1)
df['Avg_Poss_season'] = df.groupby(['Season'])['Avg_Poss_season'].expanding(1).mean().reset_index([0],drop=True)

#Standardisation
df['Avg_GF_season'] = (df['Avg_GF_season'] - df.groupby(['Season'])['Avg_GF_season'].transform(min))/(df.groupby(['Season'])['Avg_GF_season'].transform(max) - df.groupby(['Season'])['Avg_GF_season'].transform(min))
df['Avg_GA_season'] = (df['Avg_GA_season'] - df.groupby(['Season'])['Avg_GA_season'].transform(min))/(df.groupby(['Season'])['Avg_GA_season'].transform(max) - df.groupby(['Season'])['Avg_GA_season'].transform(min))
df['Avg_xG_season'] = (df['Avg_xG_season'] - df.groupby(['Season'])['Avg_xG_season'].transform(min))/(df.groupby(['Season'])['Avg_xG_season'].transform(max) - df.groupby(['Season'])['Avg_xG_season'].transform(min))
df['Avg_xGA_season'] = (df['Avg_xGA_season'] - df.groupby(['Season'])['Avg_xGA_season'].transform(min))/(df.groupby(['Season'])['Avg_xGA_season'].transform(max) - df.groupby(['Season'])['Avg_xGA_season'].transform(min))
df['Avg_Poss_season'] = (df['Avg_Poss_season'] - df.groupby(['Season'])['Avg_Poss_season'].transform(min))/(df.groupby(['Season'])['Avg_Poss_season'].transform(max) - df.groupby(['Season'])['Avg_Poss_season'].transform(min))

##AGAINST OPPONENT##
df['Avg_GF_Opp'] = df.groupby(['Club','Opp'])['GF_x'].shift(1)
df['Avg_GF_Opp'] = df.groupby(['Club','Opp'])['Avg_GF_Opp'].rolling(2).mean().reset_index([0,1],drop=True)
df['Avg_GA_Opp'] = df.groupby(['Club','Opp'])['GA_x'].shift(1)
df['Avg_GA_Opp'] = df.groupby(['Club','Opp'])['Avg_GA_Opp'].rolling(2).mean().reset_index([0,1],drop=True)
df['Avg_xG_Opp'] = df.groupby(['Club','Opp'])['xG_x'].shift(1)
df['Avg_xG_Opp'] = df.groupby(['Club','Opp'])['Avg_xG_Opp'].rolling(2).mean().reset_index([0,1],drop=True)
df['Avg_xGA_Opp'] = df.groupby(['Club','Opp'])['xGA_x'].shift(1)
df['Avg_xGA_Opp'] = df.groupby(['Club','Opp'])['Avg_xGA_Opp'].rolling(2).mean().reset_index([0,1],drop=True)
df['Avg_Poss_Opp'] = df.groupby(['Club','Opp'])['Poss_x'].shift(1)
df['Avg_Poss_Opp'] = df.groupby(['Club','Opp'])['Avg_Poss_Opp'].rolling(2).mean().reset_index([0,1],drop=True)

#Standardisation
df['Avg_GF_Opp'] = (df['Avg_GF_Opp'] - df.groupby(['Opp'])['Avg_GF_Opp'].transform(min))/(df.groupby(['Opp'])['Avg_GF_Opp'].transform(max) - df.groupby(['Opp'])['Avg_GF_Opp'].transform(min))
df['Avg_GA_Opp'] = (df['Avg_GA_Opp'] - df.groupby(['Opp'])['Avg_GA_Opp'].transform(min))/(df.groupby(['Opp'])['Avg_GA_Opp'].transform(max) - df.groupby(['Opp'])['Avg_GA_Opp'].transform(min))
df['Avg_xG_Opp'] = (df['Avg_xG_Opp'] - df.groupby(['Opp'])['Avg_xG_Opp'].transform(min))/(df.groupby(['Opp'])['Avg_xG_Opp'].transform(max) - df.groupby(['Opp'])['Avg_xG_Opp'].transform(min))
df['Avg_xGA_Opp'] = (df['Avg_xGA_Opp'] - df.groupby(['Opp'])['Avg_xGA_Opp'].transform(min))/(df.groupby(['Opp'])['Avg_xGA_Opp'].transform(max) - df.groupby(['Opp'])['Avg_xGA_Opp'].transform(min))
df['Avg_Poss_Opp'] = (df['Avg_Poss_Opp'] - df.groupby(['Opp'])['Avg_Poss_Opp'].transform(min))/(df.groupby(['Opp'])['Avg_Poss_Opp'].transform(max) - df.groupby(['Opp'])['Avg_Poss_Opp'].transform(min))

In [7]:
df_test = df[df['Date'] < np.datetime64('today')]
df_pred = df[df['Date'] >  np.datetime64('today')]
df_poo = df_pred

In [8]:
unwanted_columns = ['Date','Opp','GF_x','GA_x', 'xG_x', 'xGA_x',
       'Poss_x', 'Club', 'Poss_y', 'Position_x', 'Pl_x', 'Pts_x', 'Form_x',
       'Position_y', 'Pl_y', 'Pts_y', 'Form_y', 'spi_x', 'off_x', 'def_x',
       'Season', 'spi_y', 'off_y','def_y']
df_test = df_test.drop(columns = unwanted_columns)
df_pred = df_pred.drop(columns = unwanted_columns)
df_test = df_test.dropna()
df_pred = df_pred.dropna()


In [9]:
df_test.columns
df_test= pd.get_dummies(df_test, columns=['Venue'])
df_pred= pd.get_dummies(df_pred, columns=['Venue'])

In [10]:
boo = df_test.drop(columns = 'Result')

In [11]:
X_train = df_test[boo.columns]
y_train = df_test['Result']

X_test = df_pred[boo.columns]
y_test = df_pred['Result']

In [12]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(multi_class='multinomial', solver ='lbfgs',max_iter = 1000, penalty='l2', C=10)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [13]:
a = pd.DataFrame(logreg.predict_proba(X_test), columns = ['L', 'D', 'W'], index = X_test.index)
a['Pred_Result'] = a[['L', 'D', 'W']].max(axis=1)
df_poo = df_poo[['Season','Club','Opp','Venue']]

b = pd.merge(df_poo,a, left_index=True, right_index=True)

# display(b[b['Season']==2022])
# b[b['Season']==2022].to_csv('test.csv')
c = pd.merge(b,b, left_on = 'Club', right_on = 'Opp' )
c['L'] = (c['L_x'] + c['W_y'])/2
c['D'] = (c['D_x'] + c['D_y'])/2
c['W'] = (c['W_x'] + c['L_y'])/2
c = c[['Season_x','Club_x','Opp_x','Venue_x','W','D','L']]
c = c.rename(columns = {'Season_x':'Season','Club_x':'Club','Opp_x':'Opp','Venue_x':'Venue'})

def who_wins(x):
    if max(x['W'],x['D'],x['L']) == x['W']:
        val= x['Club'] + ' W'
    elif max(x['W'],x['D'],x['L']) == x['D']:
        val= 'D'
    else: 
        val = x['Club'] + ' L'
    return val

c['Pred_Result'] = c.apply(who_wins, axis = 1) 
# c['Nearest_Prob_Diff'] = c[['L', 'D', 'W']].max(axis = 1) - c[['L', 'D', 'W']].median(axis = 1) 
# c.to_csv(fr'{dir_path}\ML_models\{Season}_Matchweek_{Matchweek + 1}_Predictions.csv')
display(c)

,Season,Club,Opp,Venue,W,D,L,Pred_Result
0,2022,ARS,FUL,Away,0.690012,0.174537,0.135451,ARS W
1,2022,AVL,WHU,Away,0.273601,0.246587,0.479812,AVL L
2,2022,BHA,LEE,Away,0.692744,0.173735,0.133520,BHA W
3,2022,BOU,LIV,Home,0.041077,0.106254,0.852668,BOU L
4,2022,BRE,EVE,Away,0.542014,0.218637,0.239349,BRE W
5,2022,CHE,LEI,Away,0.593978,0.191055,0.214967,CHE W
6,2022,CRY,MCI,Home,0.081494,0.136406,0.782100,CRY L
7,2022,EVE,BRE,Home,0.239349,0.218637,0.542014,EVE L
8,2022,FUL,ARS,Home,0.135451,0.174537,0.690012,FUL L
9,2022,LEE,BHA,Home,0.133520,0.173735,0.692744,LEE L


In [14]:
if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

os.chdir(fr'{dir_path}\{current_season}_{current_season + 1}_Match_Predictions')

today = date.today()

c.to_csv(fr'{current_season}_{current_season + 1}_{today.strftime("%b%d")}_match_WDL_predictions.csv')